In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel
from google.colab import drive, files
import torch
import time
import gc
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

import os
import pandas as pd

from google.colab import auth
import gspread
from google.auth import default

In [ ]:
torch.cuda.empty_cache()

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_dir = "drive/My Drive/Colab Notebooks/"
file_location = "wh_island.csv"
construction = file_location[0:-4]
print(construction)

wh_island


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt2', bos_token='<sos>', eos_token='<eos>', pad_token='<pad>')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
optimizer = torch.optim.Adam(model.parameters(),lr = 0.00000625)
num_epochs = 2

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def tokenize_seq(sent,tokenizer):
  return tokenizer('<sos>'+ sent + '<eos>', truncation=True, padding="max_length")

In [ ]:
class GrammarDataset(Dataset):
    def __init__(self, data_file, tokenizer):
        dataset = pd.read_csv(data_file).iloc[0:102]   #use either 250 rows or 102 rows
        #obj_that = dataset["obj_that"]
        #obj_wh = dataset["obj_wh"]
        #pp_wh = dataset["pp_wh"]
        #subj_wh = dataset["subj_wh"]

        emb_that = dataset["emb_that"]
        emb_wh = dataset["emb_wh"]
        emb_whether = dataset["emb_whether"]
        
        #adj_that = dataset["adj_that"]
        #adj_wh = dataset["adj_wh"]
        #adj_subj_that = dataset["adj_subj_that"]
        #adj_subj_wh = dataset["adj_subj_wh"]

        #np_that = dataset["np_that"]
        #np_wh = dataset["np_wh"]

        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []
        
        #self.compiled_dataset = pd.concat([obj_that, obj_wh, pp_wh, subj_wh]).tolist()
        self.compiled_dataset = pd.concat([emb_that, emb_wh, emb_whether]).tolist()
        #self.compiled_dataset = pd.concat([adj_that, adj_wh, adj_subj_that, adj_subj_wh]).tolist()
        #self.compiled_dataset = pd.concat([np_that, np_wh]).tolist()
        #self.compiled_dataset = pd.concat([obj_that, obj_wh, pp_wh, subj_wh, emb_that, emb_wh, emb_whether, adj_that,adj_wh, adj_subj_that, adj_subj_wh, np_that, np_wh]).tolist()

        for sentence in self.compiled_dataset:
            encodings = tokenize_seq(sentence, tokenizer)
            self.input_ids.append(torch.tensor(encodings['input_ids']))
            self.attn_masks.append(torch.tensor(encodings['attention_mask']))

        self.encoded_dataset = tokenizer(self.compiled_dataset)["input_ids"]

    def __getitem__(self, index):
        return self.input_ids[index], self.attn_masks[index]   

    def __len__(self):
        return len(self.input_ids)

In [ ]:
gdata = GrammarDataset(data_file=root_dir+file_location, tokenizer=tokenizer)
train_size = int(0.9 * len(gdata))
val_size = len(gdata) - train_size
train_set, val_set = random_split(gdata, [train_size, val_size])
print("train_size :",train_size)
print("val_size   :",val_size)

gc.collect() 

print(gdata.input_ids)
print(gdata.encoded_dataset)

train_size : 275
val_size   : 31
[tensor([50257, 26886,   481,  ..., 50259, 50259, 50259]), tensor([50257, 40756,   268,  ..., 50259, 50259, 50259]), tensor([50257,  4366, 17460,  ..., 50259, 50259, 50259]), tensor([50257, 19040,  8020,  ..., 50259, 50259, 50259]), tensor([50257, 13448,  2492,  ..., 50259, 50259, 50259]), tensor([50257,   464,  2444,  ..., 50259, 50259, 50259]), tensor([50257,   464, 14625,  ..., 50259, 50259, 50259]), tensor([50257, 40797,   857,  ..., 50259, 50259, 50259]), tensor([50257, 16784,  1422,  ..., 50259, 50259, 50259]), tensor([50257,  4366, 10650,  ..., 50259, 50259, 50259]), tensor([50257, 46325,  3636,  ..., 50259, 50259, 50259]), tensor([50257, 27917,  1681,  ..., 50259, 50259, 50259]), tensor([50257,  7554, 18140,  ..., 50259, 50259, 50259]), tensor([50257,  6943,  2706,  ..., 50259, 50259, 50259]), tensor([50257, 20508,  2125,  ..., 50259, 50259, 50259]), tensor([50257,  6943,  7799,  ..., 50259, 50259, 50259]), tensor([50257,  9627,  9732,  ..., 502

In [ ]:
train_dataloader = DataLoader(train_set,  sampler = RandomSampler(train_set), batch_size = 2)
validation_dataloader = DataLoader(val_set, sampler = SequentialSampler(val_set), batch_size = 2)
print(train_dataloader)

In [ ]:
assert torch.cuda.is_available()

device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")
model.cuda()

Found device: Tesla T4, n_gpu: 1


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)

In [ ]:
def train():
    model.train()
    total_train_loss = 0

    for step, batch in enumerate(tqdm(train_dataloader)):
        input_ids = batch[0].to(device)
        masks = batch[1].to(device)
        labels = batch[0].to(device)  

        outputs = model(input_ids, attention_mask=masks, labels=labels)
        loss = outputs[0]  
        batch_loss = loss.item()
        total_train_loss += batch_loss

        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_dataloader)  
    print("\navg_train_loss ",avg_train_loss)  

In [ ]:
def eval():
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in tqdm(validation_dataloader):            
        
        with torch.no_grad():    
            input_ids = batch[0].to(device)
            masks = batch[1].to(device)
            labels = batch[0].to(device)    

            outputs  = model(input_ids,  attention_mask=masks, labels=labels)
            loss = outputs[0]              
            batch_loss = loss.item()
            total_eval_loss += batch_loss  

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    print("\navg_val_loss",avg_val_loss) 

In [ ]:
eval()

100%|██████████| 50/50 [00:12<00:00,  4.13it/s]


avg_val_loss 2.8800835943222047


In [ ]:
for e in range(num_epochs):
    train()
    eval()
    model.save_pretrained(root_dir + "GPT2_checkpoints/"+construction+"/"f"model_{e}_lr_.00000625_102")

100%|██████████| 138/138 [01:41<00:00,  1.36it/s]



avg_train_loss  0.10922153914968173


100%|██████████| 16/16 [00:04<00:00,  3.76it/s]



avg_val_loss 0.10313142370432615


100%|██████████| 138/138 [01:40<00:00,  1.37it/s]



avg_train_loss  0.09679831148705621


100%|██████████| 16/16 [00:04<00:00,  3.81it/s]



avg_val_loss 0.08721059514209628
